# DATS6103 - Individual Project 3 - Andrea Piolini 

## Introduction

This project is a data mining analysis of the H1B visa applications that were submitted by employers located in the United States between 2011 and 2016. H1B visa is an employment-based, non-immigrant visa for temporary workers in the United States. For this visa, an employer must offer the applicant a job and apply for the H1B visa petition with the US Immigration Department. Once the applicant has been sponsored by the company, the company needs to file the Labor Condition Application (LCA), which needs to be approved by United States Citizenship and Immigration Services (USCIS). If the USCIS approves it, the applicant needs to go through a lottery, and if he/she passes the lottery, the USCIS needs to review the application before approval. Once the USCIS approves the application, the applicant gets the visa. This dataset contains all the H1B Visa application submitted through the LCA to the USCIS between 2011 and 2016. If the case status says "Certified", it means that the applicant was approved to go through the lottery. However, this does not mean that the applicant eventually got the visa, and the dataset does not contain such information. 

The dataset was retrieved from Kaggle in November 2019, and it was originally scraped from the website of the U.S. Office of Foreign Labor by a Kaggle user. The dataset contains over 3 million rows and 11 columns, which include the job title, the name of the company that sponsored the visa, and its geographic coordinates. The dataset is rather thorough and does not have many missing values. However, it would have been more interesting if it included the data from 2017 and 2018 as well, as it would have been interesting to asses the impact of the Trump administration on the visa application process. 

As the number of observations is huge, my computer would take a lot of time to run the code and display the graphs. Therefore, most of the analysis was conducted on a random sample of 250,000 observations. The part on Data Science is the only part were the full dataset was analyzed.  

### Project's Goal and Structure

This analysis seek to answer the following questions:

- What were the most sponsored jobs between 2011 and 2016?
- What was the distribution of the salaries of the jobs sponsored?
- What were the companies that sponsored the largest number of visas?
- What were the cities and the states that sponsored more visas?

Moreover, as I am a foreign student studying Data Science in the United States, I decided to narrow down the scope of the analysis and just focus on the Data Science visa applications. The final section of this project seeks to answer the questions stated above for the Data Science industry only. 

This project is divided in the following sections:

- Data Cleaning Process and Preliminary Analysis
- Part 1: Most Sponsored Jobs
- Part 2: Salary Distribution
- Part 3: Companies that Sponsored the Largest Number of Visas
- Part 4: Sponsored Visas by City and State
- Part 5: Analysis of the Data Science Industry
- Part 6: Conclusion

## Data Cleaning Process and Preliminary Analysis

In [ ]:
#importing the libraries
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
import warnings
import seaborn as sns
from datetime import datetime as dt
warnings.filterwarnings('ignore')

In [ ]:
#loading the data
H1B_1 = pd.read_excel('H1B1.xlsx')

In [ ]:
H1B_1.head()

In [ ]:
H1B_1.tail()

In [ ]:
H1B_2 = pd.read_excel('H1B2.xlsx')

In [ ]:
H1B_2.head()

In [ ]:
H1B_2.tail()

In [ ]:
H1B_3 = pd.read_excel('H1B3.xlsx')

In [ ]:
H1B_3.head()

In [ ]:
#concatenating the dataframes to create a single dataset 
frames = [H1B_1, H1B_2, H1B_3]
H1B_Visas = pd.concat(frames)

In [ ]:
#looking at datatypes
H1B_Visas.dtypes

In [ ]:
#getting the shape of the data
H1B_Visas.shape

In [ ]:
#cleaning the data a little bit
H1B_Visas.columns = map(str.lower, H1B_Visas.columns) #converting al the column names to lower case
H1B_Visas.columns = map(str.title, H1B_Visas.columns) #capitalizing the first letter of each name
H1B_Visas = H1B_Visas.drop('App_Number', axis = 1)
H1B_Visas

In [ ]:
#converting all the str observations from upper case to camel case
H1B_Visas["Case_Status"] = H1B_Visas["Case_Status"].str.title()
H1B_Visas["Employer_Name"] = H1B_Visas["Employer_Name"].str.title()
H1B_Visas["Soc_Name"] = H1B_Visas["Soc_Name"].str.title()
H1B_Visas["Job_Title"] = H1B_Visas["Job_Title"].str.title()
H1B_Visas["Worksite"] = H1B_Visas["Worksite"].str.title()
H1B_Visas

In [ ]:
#checking the number of NAs
H1B_Visas.isna().sum()

In [ ]:
#dropping NAs
H1B_Visas = H1B_Visas.dropna()
H1B_Visas.shape

In [ ]:
#splitting the column Workiste into the city and the state where the company that sponsored the visa is based 
H1B_Visas[['Worksite','State']] = H1B_Visas.Worksite.str.split(',', expand=True) 
H1B_Visas

In [ ]:
#changing the Year format from float to datetime
H1B_Visas['Year'] = H1B_Visas['Year'].astype(int)
H1B_Visas.Year = pd.to_datetime(H1B_Visas.Year, format='%Y')
H1B_Visas['Year'] = H1B_Visas['Year'].dt.year

In [ ]:
#moving the State column next to the Workiste column for clarity 
H1B_Visas = H1B_Visas[['Case_Status', 'Employer_Name', 'Soc_Name', 'Job_Title', 'Full_Time_Position', 'Prevailing_Wage', 'Year', 'Worksite', 'State', 'Lon', 'Lat']]
H1B_Visas.head()

In [ ]:
#displaying descriptive statistics of the data frame
pd.set_option('display.float_format', lambda x: '%.5f' % x) #suppressing scientific notation
H1B_Visas.describe()

It seems the maximum salary in the dataset is over 6 billion dollars. This seems quite unlikely, so we are going to explore the data frame a little more to see if there are any other outliers. 

In [ ]:
#displaying all the rows with a salary higher than $1 million 
Outliers = H1B_Visas.loc[H1B_Visas['Prevailing_Wage'] >= 1000000]
Outliers

In [ ]:
#counting the number of rows with a salary higher than $1 million
Outliers.shape[0]

In [ ]:
#also checking salaries lower than $10,000 a year
Outliers1 = H1B_Visas.loc[H1B_Visas['Prevailing_Wage'] < 10000]
Outliers1

In [ ]:
#counting the number of rows with salaries lower than $10,000 a year
Outliers1.shape[0]

In [ ]:
#removing the outliers
H1B_Visas = H1B_Visas[H1B_Visas.Prevailing_Wage < 1000000]
H1B_Visas = H1B_Visas[H1B_Visas.Prevailing_Wage > 10000]
H1B_Visas.describe()

### Word Cloud

As a first step of the preliminary analysis, we will display some wordclouds to show the most frequent words in the following columns: Job_Title, Worksite, State, and Employer_Name. 

In [ ]:
#importing the necessary modules
from wordcloud import WordCloud, STOPWORDS
import numpy as np
from PIL import Image

In [ ]:
#selecting a radom sample of 250,000 observation to conduct the analysis on
H1BSample = H1B_Visas.sample(250000)
H1BSample.head()

In [ ]:
# creating a word cloud with the most common words for jobs
text = " ".join(title for title in H1BSample.Job_Title)
stopwords = set(STOPWORDS)

# Creating a word cloud image:
wordcloud = WordCloud(width = 1500, height = 800,       #setting the features of the word cloud
                background_color ='pink', 
                stopwords = stopwords, 
                min_font_size = 10).generate(text)

# Displaing the generated word cloud:
plt.figure(figsize = (15, 8), facecolor = None) 
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# creating a word cloud with the most common words for locations
text = " ".join(worksite for worksite in H1BSample.Worksite)

# Creating a word cloud image:
wordcloud = WordCloud(width = 1500, height = 800,       #setting the features of the word cloud
                background_color ='pink', 
                stopwords = stopwords, 
                min_font_size = 10).generate(text)

#Displaing the generated word cloud:
plt.figure(figsize = (15, 8), facecolor = None) 
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
#creating a word cloud with the most common words for states
text = " ".join(state for state in H1BSample.State)

#Creating a word cloud image:
wordcloud = WordCloud(width = 1500, height = 800,       #setting the features of the word cloud
                background_color ='pink', 
                stopwords = stopwords,
                max_words = 40,
                min_font_size = 10).generate(text)

#Displaing the generated word cloud:
plt.figure(figsize = (15, 8), facecolor = None) 
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# creating a word cloud with the most common words for employers
text = " ".join(employer for employer in H1BSample.Employer_Name)
stopwords.update(["Inc", "Corporation", "llc", "llp", "Limited"]) #updating stop words with corporate suffixes as they are
                                                                  #probably the most common words for companies


# Creating a word cloud image:
wordcloud = WordCloud(width = 1500, height = 800,       #setting the features of the word cloud
                background_color ='pink', 
                stopwords = stopwords, 
                min_font_size = 10).generate(text)

# Displaing the generated word cloud:
plt.figure(figsize = (15, 8), facecolor = None) 
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

Then we compare the number of approved (Certified) or denied visas. The status Certified-Withdrawn means that the application was approved by USCIS but then the employer withdrew it and decided not to go on with it. 

In [ ]:
#signing in with Plotly token
py.sign_in('AndrePiolini', '7WymlJ4rlRgAoVzL8KNI')

In [ ]:
#creating a frequency table that displays the final status of the visas
FinalStatus = H1BSample.Case_Status.value_counts()
FinalStatus = pd.DataFrame(FinalStatus)
FinalStatus

In [ ]:
FinalStatus = FinalStatus[(FinalStatus.T != 1).any()] #eliminating the 0s so they won't appear in the pie chart later

In [ ]:
#plotting the frequency table on the status of the visas using a pie chart 
colors = ['Green', 'Orange', 'Red', 'Grey']
FinalStatus.plot(kind='pie', colors = colors, figsize = (9,9), shadow = True, startangle = 140, 
         autopct='%1.1f%%', subplots=True, labels=None, wedgeprops={'linewidth' : 2, 'edgecolor' : "white"})
labels = ['Certified', 'Certified-Withdrawn', 'Denied', 'Withdrawn']
plt.title('Case Status', fontsize=14)
plt.legend(labels)
plt.ylabel('')
plt.show()

In [ ]:
#creating a frequency table that displays the case statuses per year
StatusByYear = pd.crosstab(H1BSample['Year'], H1BSample['Case_Status'])
StatusByYear = pd.DataFrame(StatusByYear)
StatusByYear

In [ ]:
#plotting case statuses per year
colors = ['Green', 'Orange', 'Red', 'Grey']
ax = StatusByYear.plot(kind = 'bar', color = colors, figsize= (20,10), fontsize = 13)
plt.legend(loc='best', fontsize = 13)
ax.set_xlabel('Year', fontsize = 13)
ax.set_ylabel('Number of Applications', fontsize = 13)
plt.xticks(rotation=360)
plt.title('Insert Title', fontsize = 16)
plt.show()

## Part 1: Most Sponsored Jobs

In [ ]:
#taking a look at the Soc_Name column
pd.set_option('display.max_rows', 2000)
SocNames = H1BSample.groupby('Soc_Name').size()
SocNames

In [ ]:
#taking a look at the Job_Title column
JobTitles = H1BSample.groupby('Job_Title').size()
JobTitles

It looks like there are over 200.000 different jobs titles, and most of them are the same but spelled differently so Python displays them as if they were different job titles. Considering the low quality of the values in this column, it would be pretty hard to gain significant insights from it. Moreover, considering the huge number of values, cleaning the data would be extremely time consuming. Therefore, in this project I'll only consider the column Soc_Name.  

In [ ]:
#creating a frequency table that displays the number of times each job got sponsored. Then converting it to a dataframe. 
SponsoredFreq = H1BSample.Soc_Name.value_counts()
SponsoredFreq = pd.DataFrame(SponsoredFreq)
SponsoredFreq

In [ ]:
#creating a dataframe with the top 20 sponsored jobs and plotting it
SponsoredFreq = SponsoredFreq.sort_values(by = 'Soc_Name')
Top20 = SponsoredFreq.tail(20)
Top20

In [ ]:
#plotting the top 20 jobs sponsored 
ax = Top20.plot(kind = 'barh', color = 'coral', figsize= (20,10), fontsize = 13, legend = None)
ax.set_xlabel('Number of Jobs Sponsored', fontsize = 13)
ax.set_ylabel('Type of Job', fontsize = 13)
plt.title('Most Popular Sponsored Jobs', fontsize = 16)
plt.show()

## Part 2: Salary Distribuition

In [ ]:
#converting the values in the Prevailing_Wage column from float to int
H1B_Visas['Prevailing_Wage'] = H1B_Visas['Prevailing_Wage'].astype(int)

In [ ]:
#displaying the caracteristics of the Prevailing_Wage column
H1BSample['Prevailing_Wage'].describe()

In [ ]:
#plotting an histogram of the prevailing wages
H1BSample['Prevailing_Wage'].plot(kind = 'hist', color = 'darkorange', bins = 100, figsize=(13,8), edgecolor='black')
plt.xlabel('Prevailing Wages',  fontsize = 13)
plt.ylabel('Number of Jobs',  fontsize = 13)
plt.title('Histogram of Wages from 2011 to 2016',  fontsize = 16)
plt.xticks(np.arange(0, 300000, step= 25000))
plt.xlim(0, 300000)
plt.show()

In [ ]:
#plotting a violin plot to see the distribution of the salaries over the years 
plt.figure(figsize=(13,8))

sns.violinplot( x=H1BSample["Prevailing_Wage"], y=H1BSample["Year"], orient = 'h', palette = "Oranges_r", inner = "quartiles")
sns.set_style(style = 'dark') #setting the style of the grid
plt.xlabel('Prevailing Wages',  fontsize = 13)
plt.ylabel('Year',  fontsize = 13)
plt.title('Wages Distribution per Year',  fontsize = 16)
plt.xticks(np.arange(0, 300000, step= 25000))
plt.xlim(0, 300000)
plt.show()

## Part 3: Companies that Sponsored the Largest Number of Visas


In [ ]:
#creating a frequency table that displays the companies that sponsored the largest number of visas. 
#Then converting it to a dataframe. 
Companies = H1BSample.Employer_Name.value_counts()
Companies = pd.DataFrame(Companies)
Companies

In [ ]:
#getting the top 20 companies
Companies = Companies.sort_values(by = 'Employer_Name')
Top20Companies = Companies.tail(20)
Top20Companies

In [ ]:
#plotting the top 20 companies
ax = Top20Companies.plot(kind = 'barh', color = 'gold', figsize= (20,10), fontsize = 13, legend = None)
ax.set_xlabel('Number of Jobs Sponsored', fontsize = 16)
ax.set_ylabel('Company', fontsize = 16)
plt.title('Companies that Sponsored the Largest Number of Visas', fontsize = 20)
plt.show()

## Part 4: Sponsored Visas by City and State

In [ ]:
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px

In [ ]:
#creating a frequency table that displays the cities where the largest number of visas was sponsored.
#Then converting it to a dataframe. 
Cities = H1BSample.Worksite.value_counts()
Cities = pd.DataFrame(Cities)
Cities

In [ ]:
#getting the top 20 cities
Cities = Cities.sort_values(by = 'Worksite')
Top20Cities = Cities.tail(20)
Top20Cities

In [ ]:
#plotting the top 20 cities
ax = Top20Cities.plot(kind = 'barh', color = 'goldenrod', figsize= (20,10), fontsize = 13, legend = None)
ax.set_xlabel('Number of Jobs Sponsored', fontsize = 16)
ax.set_ylabel('City', fontsize = 16)
plt.title('Cities that Sponsored the Largest Number of Visas', fontsize = 20)
plt.show()

In [ ]:
#plotting every single visa sponsored on a map of the continental United States. Every dot indicates the company that sponsored the visa, 
#the job title, the year and the city where the visa was sponsored
fig = px.scatter_mapbox(H1BSample, lat="Lat", lon="Lon", hover_name="Employer_Name", hover_data=["Case_Status", "Job_Title", "Year", "Worksite"],
                        color_discrete_sequence=["orange"], zoom=3, height=500)
#setting the type of map
fig.update_layout(mapbox_style="carto-darkmatter")
#setting the map's margins
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
#creating a new column called Number of Visas Sponsored
H1BSample['Number of Visas Sponsored'] = 1

In [ ]:
#plotting a density map displaying the number of visas sponsored
fig = go.Figure(go.Densitymapbox(lat=H1BSample.Lat, lon=H1BSample.Lon, z=H1BSample['Number of Visas Sponsored'], 
                                 radius=10, zmin = 0, zmax = 500))
#setting the type of map, the center and the zoom level
fig.update_layout(mapbox_style="open-street-map", mapbox_center_lon=-96.683421, mapbox_center_lat = 39.911756, mapbox_zoom=3)
#setting the map's margins
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
#uploading the dataset StatFIPS. This dataset was retrieved from one of the lectures and 
#it will be useful to build a chloropleth map
StateFIPS = pd.read_csv("StateFIPS.csv", dtype={'statefips':str})
StateFIPS.rename(columns = {StateFIPS.columns[0]: "State"}, inplace = True)
StateFIPS.head()

In [ ]:
H1BSample.State = H1BSample.State.astype(str) #making sure the State columns for both data frames are in str format
StateFIPS.State = StateFIPS.State.astype(str)
H1BSample['State'] = H1BSample['State'].str.strip() #removing any unnecessary space in the State colums for both data frame. 
StateFIPS['State'] = StateFIPS['State'].str.strip() #unnecessray spaces will prevent the two data frames from merging properly

In [ ]:
#merging the two data frames
H1BFips = pd.merge(H1BSample, StateFIPS, on='State')
H1BFips = pd.DataFrame(H1BFips)
H1BFips.head()

In [ ]:
#checking the shape of H1BFips
H1BFips.shape

In [ ]:
#creating a new data frame ByState showing the number of sponsorizations by state abreviation
ByState = H1BFips.groupby('stateabrev').sum()
ByState = pd.DataFrame(ByState)
ByState.reset_index(level=0, inplace=True) #transforming the index colum stateabrev into a normal column
ByState

In [ ]:
#creating a chloropleth map displaying the number of visas sponsored by state
fig = go.Figure(data=go.Choropleth(
    locations= ByState['stateabrev'], # Spatial coordinates
    z = ByState['Number of Visas Sponsored'], # Data to be color-coded
    locationmode = 'USA-states', 
    colorscale = 'Oranges',
    autocolorscale=False,
    colorbar_title = "Number of Visas Sponsored",
    zmin = 0, zmax = 20000,
))

fig.update_layout(
    title_text = 'Number of Visas Sponsored by State',
    geo_scope='usa', # limite map scope to the US
)

fig.show()

## Part 5: Analysis of the Data Science Industry

Mention that this is not going to be done on a sample 

In [ ]:
#creating a new data frame that only contains jobe related to the Data Science industry by subsetting the H1B_Visas data frame
DSOthers = H1B_Visas.copy()
DataScience = H1B_Visas[H1B_Visas['Job_Title'].str.contains("Data Scientist" or "Data Analytics" or "Data Analyst")]
DataScience.head()

As a preliminary analysis, I want to see what percentage of the overall jobs sponsored data science jobs account for.

In [ ]:
#calculatingwhat percentage of the overall jobs sponsored data science jobs account for
NumberDSJobs = len(DataScience)
NumberOthers = len(H1B_Visas) - len(DataScience)
PercJob = (NumberDSJobs/NumberOthers) * 100
PercJob

In [ ]:
#creating a new column called Number of Visas Sponsored
DataScience['Number of Visas Sponsored'] = 1
DataScience.head()

In [ ]:
#creating a frequency table by year to see the number of the data science visas sponsored over the years
ByYear = pd.crosstab(DataScience['Year'], DataScience['Number of Visas Sponsored'])
ByYear

In [ ]:
#plotting the numbers of data science applications over time
ax = ByYear.plot(colors = 'SpringGreen', figsize= (13,8), fontsize = 13, linewidth = 3.0, legend = None)
ax.set_xlabel('Year', fontsize = 13)
ax.set_ylabel('Number of Applications', fontsize = 13)
plt.title('Number of Data Science Applications Over Time', fontsize = 16)
plt.show()

In [ ]:
#displaying the caracteristics of the Prevailing_Wage column for the Data Science data frame 
DataScience['Prevailing_Wage'].describe()

In [ ]:
#plotting an histogram of the prevailing wages for the Data Science data frame 
DataScience['Prevailing_Wage'].plot(kind = 'hist', color = 'SpringGreen', bins = 50, figsize=(13,8), edgecolor='black')
plt.xlabel('Prevailing Wages',  fontsize = 13)
plt.ylabel('Number of Jobs',  fontsize = 13)
plt.title('Histogram of Wages for Data Science Jobs from 2011 to 2016',  fontsize = 16)
plt.xticks(np.arange(0, 300000, step= 25000))
plt.show()

In [ ]:
#plotting a violin plot to see the distribution of the wages in Data Science over time
plt.figure(figsize=(13,8))

sns.violinplot( x=DataScience["Prevailing_Wage"], y=DataScience["Year"], orient = 'h', palette = "Greens", inner = "quartiles")
sns.set_style(style = 'dark')
plt.xlabel('Prevailing Wages',  fontsize = 13)
plt.ylabel('Year',  fontsize = 13)
plt.title('Wages Distribution per Year',  fontsize = 16)
plt.xticks(np.arange(0, 300000, step= 25000))
plt.show()

In [ ]:
#creating a frequency table that displays the Data Science companies that sponsored the largest number of visas. 
#Then converting it to a dataframe. 
DataScienceCompanies = DataScience.Employer_Name.value_counts()
DataScienceCompanies = pd.DataFrame(DataScienceCompanies)
DataScienceCompanies

In [ ]:
#getting the top 20 companies that sponsored the largest amount of visas
DataScienceCompanies = DataScienceCompanies.sort_values(by = 'Employer_Name')
Top20DSCompanies = DataScienceCompanies.tail(20)
Top20DSCompanies

In [ ]:
#plotting the top 20 companies
ax = Top20DSCompanies.plot(kind = 'barh', color = 'SpringGreen', figsize= (20,10), fontsize = 13, legend = None)
ax.set_xlabel('Number of Jobs Sponsored', fontsize = 16)
ax.set_ylabel('Company', fontsize = 16)
plt.title('Companies that Sponsored the Largest Number of Visas in the Data Science Industry', fontsize = 20)
plt.show()

The overall results of the graph displayed above make sense, as  most of the companies that sponsor the largest number of visas for data scientist are well-known, multinational tech companies. However, I was surprised Amazon is not among them. I am therefore curious to see how many visas Amazon sponsored in the data science field from 2011 and 2016. 

In [ ]:
#looking for the visas sponsored by Amazon in the data science field
DataScience.loc[DataScience['Employer_Name'].str.contains("Amazon")]

It seems that Amazon did not sponsor any visa in the data science field between 2011 and 2016. This is quite surprising, let's take a look at the jobs sponsored by Amazon:

In [ ]:
#looking for the visas sponsored by Amazon in other fields
AmazonJobs = H1B_Visas.loc[H1B_Visas['Employer_Name'].str.contains("Amazon")]
AmazonJobs = AmazonJobs.Job_Title.value_counts()
AmazonJobs = pd.DataFrame(AmazonJobs)
AmazonJobs

In [ ]:
#getting the top 20 job titles
AmazonJobs = AmazonJobs.sort_values(by = 'Job_Title')
Top20Amazon = AmazonJobs.tail(20)
Top20Amazon

In [ ]:
#plotting the top 20 job titles
ax = Top20Amazon.plot(kind = 'barh', color = 'cornflowerblue', figsize= (20,10), fontsize = 13, legend = None)
ax.set_xlabel('Number of Jobs Sponsored', fontsize = 16)
ax.set_ylabel('Job Title', fontsize = 16)
plt.title('Top 20 Job Titles Sponsored by Amazon Between 2011 and 2016', fontsize = 20)
plt.show()

It seems that the jobs that Amazon sponsored the most are software development engineers, product managers, and program managers. These two last job titles are rather vague, and data scientists might very well work as program and product managers. Consequently, we can not be extremely sure that Amazon did not sponsor data science jobs whatsoever. 

In [ ]:
#creating a frequency table that displays cities where the largest number of Data Science visas was sponsored. 
#Then converting it to a dataframe. 
DataScienceCities = DataScience.Worksite.value_counts()
DataScienceCities = pd.DataFrame(DataScienceCities)
DataScienceCities

In [ ]:
#getting the top 20 cities
DataScienceCities = DataScienceCities.sort_values(by = 'Worksite')
Top20DSCities = DataScienceCities.tail(20)
Top20DSCities

In [ ]:
#plotting the top 20 cities
ax = Top20DSCities.plot(kind = 'barh', color = 'DarkGreen', figsize= (20,10), fontsize = 13, legend = None)
ax.set_xlabel('Number of Jobs Sponsored', fontsize = 16)
ax.set_ylabel('City', fontsize = 16)
plt.title('Cities that Sponsored the Largest Number of Visas in the Data Science Industry', fontsize = 20)
plt.show()

In [ ]:
#plotting every single Data Science visa sponsored on a map of the continental United States. Every dot indicates the company that sponsored the visa, 
#the job title, the year and the city where the visa was sponsored
fig = px.scatter_mapbox(DataScience, lat="Lat", lon="Lon", hover_name="Employer_Name", hover_data=["Case_Status", "Job_Title", "Year", "Worksite"],
                        color_discrete_sequence=["SpringGreen"], zoom=3, height=500)
#setting the type of map
fig.update_layout(mapbox_style="carto-darkmatter")
#setting the map's margins
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
#plotting a density map displaying the number of Data Science visas sponsored
fig = go.Figure(go.Densitymapbox(lat=DataScience.Lat, lon=DataScience.Lon, z=DataScience['Number of Visas Sponsored'], 
                                 radius=10, zmin = 0, zmax = 10))
#setting the type of map, the center and the zoom level
fig.update_layout(mapbox_style="open-street-map", mapbox_center_lon=-96.683421, mapbox_center_lat = 39.911756, mapbox_zoom=3)
#setting the map's margins
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
#creating a new data frame that contains the state fips to make a chloropleth
DataScience.State = DataScience.State.astype(str) #deleting unnecessary spaces in the State column 
DataScience['State'] = DataScience['State'].str.strip()

#merging the two data frames
DataScienceFips = pd.merge(DataScience, StateFIPS, on='State')
DataScienceFips = pd.DataFrame(DataScienceFips)
DataScienceFips.head()

In [ ]:
#creating a new data frame DSByState showing the number of sponsorizations in the data science industry by state abreviation
DSByState = DataScienceFips.groupby('stateabrev').sum()
DSByState = pd.DataFrame(DSByState)
DSByState.reset_index(level=0, inplace=True) #transforming the index colum stateabrev into a normal column
DSByState.head()

In [ ]:
#creating a chloropleth map 
fig = go.Figure(data=go.Choropleth(
    locations= DSByState['stateabrev'], # Spatial coordinates
    z = DSByState['Number of Visas Sponsored'], # Data to be color-coded
    locationmode = 'USA-states', 
    colorscale = 'Greens',
    autocolorscale=False,
    colorbar_title = "Number of Visas Sponsored",
    zmin = 0, zmax = 300,
))

fig.update_layout(
    title_text = 'Number of Visas Sponsored in the Data Science Industry by State',
    geo_scope='usa', # limite map scope to the US
)

fig.show()

## Conclusions

### Complete Dataset

- The vast majority of the H1B Visa application were approved. Only approximately 3% of the application was denied. 
- The most sponsored jobs between 2011 and 2016 were jobs related to computers, software and technology in general. Finance-related jobs such as accountants and financial analysts were also in the top 20 most sponsored jobs. 
- The wage distribution for all jobs between 2011 and 2016 is right skewed. When looking at the wage distribution for each year, we can see that they are right skewed as well and overall rather similar to each other, although there was a slow, yet steady increase of the first quartile, median, and third quartile for each year. 
- The two companies that sponsored the largest number of visas are Infosys and Tata Consultancy Services, two Indian multinational consulting firms. Overall, consulting firms are the ones that sponsored the largest number of visas, followed by IT and tech giants such as Microsoft and Amazon. 
- Eight cities among the top 20 cities that sponsored the largest number of visas are located in California. However, New York City is the city with the highest number of applications. Big tech and business hubs such as Seattle, Houston, and Atlanta are also among the top 20 cities. 
- California, New York, and Texas are the states where the majority of the visas were sponsored. Additionally, a significant number of applications were filed in Illinois, New Jersey, and Washington state. 

### Data Science

- The applications to sponsor jobs in the data science field spiked between 2011 and 2016; however, data science jobs represented only 0.1 percent of the total jobs sponsored. This indicates that data science is becoming more popular and it is a booming field, but it is also a relatively new field and the number of data scientist is still relatively low. It would have been interesting to see if the trend was confirmed in 2017 and 2018. Most likely it was. 
- The average salary of a data scientist is higher than the average salary of the total jobs sponsored.
- The companies that sponsored the largest number of data science jobs are well-known, tech giants such as Microsoft, Uber, and Facebook. 
- Eleven cities out of the top 20 that sponsored the largest number of visas in data science are located in California. Unsurprisingly then, California is the state that sponsored most visas in the Data Science industry. Other states that sponsored quite a lot in the industry are New York, Washington, Illinois, and Texas. 

### 2017-2018 Trends

As stated before, one limitation of this dataset is that it does not contain the data for 2017 and 2018. However, I doubt that the major trends we saw in this analysis changed. The most sponsored jobs are most likely the ones related to software development, computers and consulting, and the companies that sponsor the majority of the visas are probably multinational companies operating in consulting, IT, and technology in general. Location-wise, the situation has probably not changed either. Cities like San Francisco, Seattle, and New York City are still major business and technology hubs where companies keep sponsoring a significant amount of visas every year. The trends in the Data Science industry most likely remained the same through 2017 and 2018. The only thing that might have changed significantly is the number of the application for Data Science, which I expect to have grown tremendously in the last two years. 

Overall, the only major difference between the applications filed between 2011 and 2016 and the ones filed in 2017-2018 might be the denial rate. Indeed - according to several media outlets - the Trump Administration reportedly has been "aggressively" denying applications for H1B visas since Trump took office in January 2017. 